In [1]:
import plotly.express as px
import pandas as pd
import config
from web.shared_utils import load_data

2025-10-15 00:19:07.970 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-15 00:19:07.971 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-15 00:19:07.971 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-15 00:19:07.972 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-15 00:19:07.972 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-15 00:19:07.972 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-15 00:19:07.973 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [2]:
keywords, grants, cats = load_data()

2025-10-15 00:19:15.057 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-15 00:19:15.058 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-15 00:19:15.184 
  command:

    streamlit run /Users/luhancheng/Desktop/research-link-technology-landscaping/modeling/.venv/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-10-15 00:19:15.184 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-15 00:19:15.184 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-15 00:19:15.184 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-15 00:19:15.743 Thread 'Thread-9': missing ScriptRunContext! This warning can be ignored when running i

In [5]:
from web.pages.comparing_organisations import get_category_grant_links

In [7]:
linked = get_category_grant_links(cats, keywords)

In [8]:
linked

,category,grant_id
0,Climate and Paleoenvironmental Reconstruction,arc/de160101020
1,Climate and Paleoenvironmental Reconstruction,arc/de130100668
2,Climate and Paleoenvironmental Reconstruction,arc/lx0212015
3,Climate and Paleoenvironmental Reconstruction,arc/dp0772938
4,Climate and Paleoenvironmental Reconstruction,arc/dp0558150
...,...,...
364868,"Defense, Security, and National Technologies",arc/lp0776636
364869,"Defense, Security, and National Technologies",arc/lp0562156
364870,"Defense, Security, and National Technologies",arc/dp0773136
364871,"Defense, Security, and National Technologies",arc/dp130102351


In [18]:
cats.explode("keywords").join(keywords.explode("grants"), on="keywords", rsuffix="_grant")

,description,field_of_research,keywords,type,description_grant,grants,organisation_ids
name,,,,,,,
Climate and Paleoenvironmental Reconstruction,A unified domain integrating climate variabili...,BIOLOGICAL_SCIENCES,Australia's flooding history,General,The geological history of coastal flooding and...,arc/de160101020,"[au:sydney_university, au:university_wollongon..."
Climate and Paleoenvironmental Reconstruction,A unified domain integrating climate variabili...,BIOLOGICAL_SCIENCES,Australian climate history,General,Focus on reconstructing and understanding Aust...,arc/de130100668,[au:melbourne_university]
Climate and Paleoenvironmental Reconstruction,A unified domain integrating climate variabili...,BIOLOGICAL_SCIENCES,Cenozoic landscape evolution,General,Long-term geological change over time.,arc/lx0212015,[au:queensland_university]
Climate and Paleoenvironmental Reconstruction,A unified domain integrating climate variabili...,BIOLOGICAL_SCIENCES,Eocene-Oligocene transition,General,A key period in Earth's climate history when t...,arc/dp0772938,[au:australia_bureau_geoscience_mineral_resour...
Climate and Paleoenvironmental Reconstruction,A unified domain integrating climate variabili...,BIOLOGICAL_SCIENCES,Eocene-Oligocene transition,General,A key period in Earth's climate history when t...,arc/dp0558150,[au:australia_bureau_geoscience_mineral_resour...
...,...,...,...,...,...,...,...
"Defense, Security, and National Technologies",A specialized domain addressing technologies w...,ENGINEERING,timely response,Application,Emphasis on reducing the time between failure ...,arc/lp0776636,"[au:queensland_university, xx:bluescope_ltd_st..."
"Defense, Security, and National Technologies",A specialized domain addressing technologies w...,ENGINEERING,traceback,Methodology,Technique to identify and locate the origin of...,arc/lp0562156,[au:burwood_campus_deakin_melbourne_university...
"Defense, Security, and National Technologies",A specialized domain addressing technologies w...,ENGINEERING,trusted mobile interaction,Technology,Core research focus on enabling secure and rel...,arc/dp0773136,"[au:sydney_technology_university, au:sydney_un..."


In [12]:
keywords.explode("grants")

,type,description,grants,organisation_ids
name,,,,
chronic pain,Application,A potential clinical application of the resear...,arc/lp0668324,"[au:adelaide_university, au:alfred_health, au:..."
chronic pain,Application,A potential clinical application of the resear...,nhmrc/2007062,"[au:adelaide_university, au:alfred_health, au:..."
chronic pain,Application,A potential clinical application of the resear...,nhmrc/1054041,"[au:adelaide_university, au:alfred_health, au:..."
chronic pain,Application,A potential clinical application of the resear...,nhmrc/1084240,"[au:adelaide_university, au:alfred_health, au:..."
chronic pain,Application,A potential clinical application of the resear...,arc/ft130100781,"[au:adelaide_university, au:alfred_health, au:..."
...,...,...,...,...
26-week,Methodology,The duration of the primary intervention phase...,johnson-and-johnson/ff-301-2010,"[au:monash_university, id:padjadjaran_universi..."
canagliflozin,Technology,A specific medication used in the study for tr...,johnson-and-johnson/ff-301-2010,"[au:monash_university, id:padjadjaran_universi..."
moderate renal impairment,Application,A specific patient population with reduced kid...,johnson-and-johnson/ff-301-2010,"[au:monash_university, id:padjadjaran_universi..."
